# Problem Set 5 (Jugal Marfatia)

## Question 2.  

In [26]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from linearmodels import PanelOLS
from linearmodels.panel import PooledOLS

df = pd.read_excel('/Users/jugalmarfatia/Downloads/jtrain1.xlsx')

## Include year 1987 and 1988 and drop null values

In [2]:
df = df[df['hrsemp'] >= 0]
df = df[df['year'] !=1989]

df.head()

,year,fcode,employ,sales,avgsal,scrap,rework,tothrs,union,grant,...,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp
0,1987,410032,100.0,47000000.0,35000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,10.463103,NaN,NaN,NaN,NaN
1,1988,410032,131.0,43000000.0,37000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,0.270027,-0.088949,10.518673,0.05557,0.0,-8.946565,-1.165385
3,1987,410440,12.0,1560000.0,10500.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,9.259130,NaN,NaN,NaN,NaN
4,1988,410440,13.0,1970000.0,11000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,0.080043,0.233347,9.305651,0.04652,0.0,0.000000,0.000000
6,1987,410495,20.0,750000.0,17680.0,NaN,NaN,50.0,0,0,...,0,NaN,0,NaN,NaN,9.780190,NaN,NaN,NaN,NaN


In [3]:
df['E'] = df.fcode.isin(df[df['grant'] ==1].fcode.tolist()).astype(int)
df['d_1988'] = (df['year'] == 1988).astype(int)

year = pd.Categorical(df.year)
fcode = pd.Categorical(df.fcode)

df = df.set_index(['fcode', 'year'])
df['year'] = year
df['fcode'] = fcode

## 2.a.i. Literal diff in diff of the means

In [4]:
x = df.groupby(['year', 'E'])['hrsemp'].mean()

(x[1988, 1] - x[1987, 1]) - (x[1988, 0] - x[1987, 0])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'year' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


28.012916087399294

## 2.a.ii

# $hrsemp_{it} = β_0 + β_1 grant_{it} + β_{2}1 (year = 1988) + β_3 E_i + u_{it} $

In [5]:
exog_vars = ['grant','d_1988', 'E']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.hrsemp, exog)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 hrsemp   R-squared:                        0.1501
Estimator:                   PanelOLS   R-squared (Between):              0.0692
No. Observations:                 256   R-squared (Within):               0.4710
Date:                Thu, Oct 04 2018   R-squared (Overall):              0.1501
Time:                        14:00:58   Log-likelihood                   -1140.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      14.831
Entities:                         131   P-value                           0.0000
Avg Obs:                       1.9542   Distribution:                   F(3,252)
Min Obs:                       1.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             14.637
                            

## 2.a.ii

# $hrsemp_{it} = \theta_i + β_1 grant_{it} + β_{2}1 (year = 1988) +  u_{it} $

In [6]:
exog_vars = ['grant', 'd_1988', 'fcode']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.hrsemp,  exog)
res = mod.fit()
print(res.params[0:3])

const      7.272056
grant     27.877932
d_1988     0.509323
Name: parameter, dtype: float64


## We get approximately the same diff in diff parameter estimates from all 3 techniques because they are theoretically equivalent. 

## 2.b Again load data and exculde null values 

In [7]:
df = pd.read_excel('/Users/jugalmarfatia/Downloads/jtrain1.xlsx')
df = df[df['hrsemp'] >= 0]
df['E'] = df.fcode.isin(df[df['grant'] ==1].fcode.tolist()).astype(int)

year = pd.Categorical(df.year)
fcode = pd.Categorical(df.fcode)

df = df.set_index(['fcode', 'year'])
df['year'] = year
df['fcode'] = fcode
df['year*fcode'] = pd.Categorical(df['year'].astype(str)+ '_' + df['fcode'].astype(str))

## 2.b.i. Run fixed effect model with time and firm dummy variables for all 3 year
# $hrsemp_{it} = β_0 + β_1 grant_{it} + \theta_i + \lambda_t + u_{it} $

In [8]:
exog_vars = ['grant', 'fcode', 'year']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.hrsemp,  exog)
res = mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 hrsemp   R-squared:                        0.8016
Estimator:                   PanelOLS   R-squared (Between):              1.0000
No. Observations:                 390   R-squared (Within):               0.4908
Date:                Thu, Oct 04 2018   R-squared (Overall):              0.8016
Time:                        14:00:58   Log-likelihood                   -1503.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      7.4337
Entities:                         135   P-value                           0.0000
Avg Obs:                       2.8889   Distribution:                 F(137,252)
Min Obs:                       1.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             7.4337
                            

## 2.b.2. Run fixed effect model with time and firm dummy variables 

# $hrsemp_{it} = β_0 + \theta_i + \lambda_t + u_{it} $ to predict $ \hat{u_{it}}$
# $grant_{it} = a_0 + \theta_i + \lambda_t + e_{it} $ to predict $ \hat{e_{it}}$

# *Running the above two regression is tehoretically equivalent to running seperate regression for each form with constant and time trends because we allow for different intercept for each firm. 

In [9]:
exog_vars = ['year', 'fcode']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.hrsemp,  exog)
res = mod.fit()
df['y_res'] = res.predict()

In [10]:
exog_vars = ['year', 'fcode']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.grant,  exog)
res = mod.fit()
df['grant_res'] = res.predict()

# $ \hat{e_{it}} = c_0 + \hat{u_{it}} $ 


In [11]:
exog_vars = ['grant_res']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.y_res,  exog)
res = mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  y_res   R-squared:                        0.0860
Estimator:                   PanelOLS   R-squared (Between):              0.0511
No. Observations:                 390   R-squared (Within):               0.7492
Date:                Thu, Oct 04 2018   R-squared (Overall):              0.0860
Time:                        14:00:58   Log-likelihood                   -1716.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      36.520
Entities:                         135   P-value                           0.0000
Avg Obs:                       2.8889   Distribution:                   F(1,388)
Min Obs:                       1.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             36.520
                            

# I get a similar estimate to the previous regression because they are theoretically equivalent. 